In [ ]:
!pip install -U pymilvus
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
from pymilvus import MilvusClient
from pymilvus import FieldSchema, DataType
from pymilvus import FieldSchema, CollectionSchema

import pandas as pd
import numpy as np
import time
import openai
from openai import OpenAI
import os

In [ ]:
# 데이터베이스 연결

client = MilvusClient("milvus_scalar_index2.db")

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: cfb7a1e2e25c4b27b4de67499343ce5f


In [ ]:
from google.colab import userdata

#openai api 쓰기 위한 환경변수 설정
EMBEDDINGS_KEY = userdata.get('EMBEDDINGS_KEY')
os.environ["OPENAI_API_KEY"] = EMBEDDINGS_KEY

openAI_api = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [ ]:
# 만들어진 컬렉션 확인
client.list_collections()

['kstartup_travel_sites', 'nature_travel_sites', 'nowlocal_travel_sites']

In [ ]:
# 데이터 가져올 파일명
data_list = {
    # 컬렉션 이름 : 가져올 데이터 파일이름
    'kstartup_travel_sites' : 'data_kstartup_time.xlsx',
    'nowlocal_travel_sites' : 'data_nowlocal_time.xlsx',
    'nature_travel_sites' : 'data_natural_attractions.xlsx'
}

In [ ]:
# 컬렉션 구체적 정보(인덱스 지정 필드, 삽입 데이터 개수) 확인

for collection_name, file_name in data_list.items():
  print(f'-- {collection_name} --')
  res = client.list_indexes(collection_name=collection_name)
  print(f'인덱스 지정된 필드 리스트: {res}')

  res = client.query(
      collection_name=collection_name,
      filter="",
      output_fields=["count(*)"]
  )
  print(f'삽입 데이터 개수: {res}')
  print()


-- kstartup_travel_sites --
인덱스 지정된 필드 리스트: ['area_name', 'embedding']
삽입 데이터 개수: data: ["{'count(*)': 244}"] , extra_info: {'cost': 0}

-- nowlocal_travel_sites --
인덱스 지정된 필드 리스트: ['area_name', 'embedding']
삽입 데이터 개수: data: ["{'count(*)': 154}"] , extra_info: {'cost': 0}

-- nature_travel_sites --
인덱스 지정된 필드 리스트: ['area_name', 'embedding']
삽입 데이터 개수: data: ["{'count(*)': 68}"] , extra_info: {'cost': 0}



In [ ]:
# ------- RAG 테스트 로직 ---------
# 컬렉션 로드하기
for collection_name, file_name in data_list.items():
    client.load_collection(
      collection_name=collection_name,
      #replica_number=1 # Number of replicas to create on query nodes. Max value is 1 for Milvus Standalone, and no greater than `queryNode.replicas` for Milvus Cluster.
  )

In [ ]:
# 컬렉션 로드 된지 확인
for collection_name, file_name in data_list.items():
  res = client.get_load_state(
    collection_name=collection_name
  )
  print(res)

{'state': <LoadState: Loaded>}
{'state': <LoadState: Loaded>}
{'state': <LoadState: Loaded>}


In [ ]:
# 받은인풋(=질문)을 임베딩 해줍니다...
def embed_question(question):
    response = openAI_api.embeddings.create(
        input=question,
        model="text-embedding-3-small",
        dimensions=768
    )
    embedding = response.data[0].embedding
    return embedding

# 행정 구역 필터링 생성 함수
def make_filtering(question):
  response = openAI_api.chat.completions.create(
      model="gpt-4o",
      messages=[
          {"role": "system", "content":"사용자의 질문 속의 장소가 아래의 한국 행정 구역 중 어디에 속하는지 다음과 같이 단어로만 대답해주세요.\n ex. 강원특별자치도 \n\n - 한국 행정 구역 : \n서울특별시, 부산광역시, 인천광역시, 대구광역시, 대전광역시, 광주광역시, 울산광역시, 세종특별자치시, 경기도, 충청북도, 충청남도, 전라남도, 경상북도, 경상남도, 강원특별자치도, 전북특별자치도, 제주특별자치도"},
          {"role": "user", "content": f"{question}"}
      ]
  )
  return response.choices[0].message.content

# 단일 테이블 검색 함수
def search_table(table_name, embedding, filtering, top_k):
    search_params = {"metric_type": "IP", "params": {}}
    results = client.search(
        collection_name=table_name,
        data=[embedding],
        filter=f"area_name == '{filtering}'",
        anns_field="embedding",
        search_params=search_params,
        output_fields=["id", "text", "operation"],
        limit=top_k
    )
    return results

# 다중 테이블 검색 함수
def search_all_tables(embedding, filtering):
    results_localCreator = search_table('kstartup_travel_sites', embedding, filtering, top_k=10)
    results_nowLocal = search_table('nowlocal_travel_sites', embedding, filtering, top_k=10)
    results_nature = search_table('nature_travel_sites', embedding, filtering, top_k=5)

    return results_localCreator, results_nowLocal, results_nature

# 쿼리 검색 결과 하나의 문자열로 묶기
# 개행으로한줄씩구분완
def format_results(results_localCreator, results_nowLocal, results_nature):
    list_of_results = [results_localCreator[0], results_nowLocal[0], results_nature[0]]
    formatted_results = ""

    for result in list_of_results:
      length = len(result)
      for num in range(length):
        text = result[num]['entity']['text']
        text += f"운영정보: '{result[num]['entity']['operation']}'"
        formatted_results += text + "\n\n"

    return formatted_results

# 프롬프트엔지니어링은각자도생
def sendLLM(question, results):
    response = openAI_api.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "- 당신은 여행을 계획하는데 도움을 주는 chatbot 'TBTI'입니다. \n- 당신의 역할은 사용자의 질문에 reference를 바탕으로 답변하는 것 입니다.\n- reference 참고 자료에서 쓸만한 정보가 부족할 때, 당신이 기존에 알고 있는 정보를 이용하여 답변하세요.\n- 장소에 대한 정보를 전달할 때는 아래의 조건을 만족해야 합니다.\n\n조건:\n1. 전달 장소 개수: 5개 \n2. 각 장소는 위치, 카테고리, 설명, 운영정보가 있어야 합니다.\n3. 설명은 장소의 키워드를 이용하여 만들어주세요. \n4.운영 정보가 존재할 때만, 운영 정보를 제공해주세요."},
            {"role": "user", "content":f"사용자 질문: {question} \n reference: {results}" }
        ]
    )
    return response.choices[0].message.content

In [ ]:
def main(question):
  # 장소 행정구역 필터링
  filtering = str(make_filtering(question))

  # 질문 임베딩
  embedding = embed_question(question)

  # 테이블 검색
  results_localCreator, results_nowLocal, results_nature = search_all_tables(embedding, filtering)

  # 쿼리 검색 결과 하나의 문자열로 묶기
  formatted_results = format_results(results_localCreator, results_nowLocal, results_nature)

  # LLM에 전달
  LLM_response = sendLLM(question, formatted_results)
  return LLM_response, formatted_results

In [ ]:
example_question = "충남 여행지 추천해줘"
#start = time.time()
#llm_response, reference = main(example_question)
#end = time.time()
#print(llm_response)

In [ ]:
# 장소 행정구역 필터링
filtering = make_filtering(example_question)
filtering

'충청남도'

In [ ]:
embedding = embed_question(example_question)
len(embedding)

In [ ]:
# 테이블 검색
results_localCreator, results_nowLocal, results_nature = search_all_tables(embedding, filtering)

print(results_localCreator)
print("---------------------------------------")
print(results_nowLocal)
print("---------------------------------------")
print(results_nature)

data: ["[{'id': 451657650232361173, 'distance': 0.5193788409233093, 'entity': {'id': 451657650232361173, 'operation': '정보가 없습니다.', 'text': '장소명: 민들레역사문화연구소 협동조합\\n카테고리: 기타\\n장소 키워드:  “랜선으로 만나는 역사 체험 키트 개발”\\n위치: 충남 아산시 배방읍 배방로 22 301호\\n해시태그: 충남, 아산시, 아산, 기타, 추천 여행지, 놀만한 곳, 가볼 만한 곳\\n'}}, {'id': 451657650232361174, 'distance': 0.48736464977264404, 'entity': {'id': 451657650232361174, 'operation': '정보가 없습니다.', 'text': '장소명: 이응팩토리\\n카테고리: 기타\\n장소 키워드:  “부여선화핸즈 백제감성체험”\\n위치: 충남 부여군 규암면 수북로 57\\n해시태그: 충남, 부여군, 부여, 기타, 추천 여행지, 놀만한 곳, 가볼 만한 곳\\n'}}, {'id': 451657650232361176, 'distance': 0.4853290319442749, 'entity': {'id': 451657650232361176, 'operation': '월~금 10:00 ~ 18:00', 'text': '장소명: 짭쪼름갱갱\\n카테고리: 기타\\n장소 키워드:  짭쪼름 갱갱\\n위치: 충남 논산시 강경읍 금백로 135 1층\\n해시태그: 충남, 논산시, 논산, 기타, 추천 여행지, 놀만한 곳, 가볼 만한 곳\\n'}}, {'id': 451657650232361171, 'distance': 0.46987277269363403, 'entity': {'id': 451657650232361171, 'operation': '정보가 없습니다.', 'text': '장소명: 엘캄포\\n카테고리: 체험\\n장소 키워드:  열풍 건조 쪽파 디저트 및 스낵\\n위치: 충

In [ ]:
# 검색 결과 확인
list_of_results = {
    "local_creator" : results_localCreator[0],
    "now_local" : results_nowLocal[0],
    "nature" : results_nature[0]
}

for table_name, table in list_of_results.items():
  print(f"{table_name}\n")
  for i in range(len(table)):
    print(f'- {i+1}번 검색 결과\n')
    print(f"distance: {table[i]['distance']} \n")
    print(table[i]['entity']['text'])
    print()
  print("----------------------------------------")


local_creator

- 1번 검색 결과

distance: 0.5193788409233093 

장소명: 민들레역사문화연구소 협동조합
카테고리: 기타
장소 키워드:  “랜선으로 만나는 역사 체험 키트 개발”
위치: 충남 아산시 배방읍 배방로 22 301호
해시태그: 충남, 아산시, 아산, 기타, 추천 여행지, 놀만한 곳, 가볼 만한 곳


- 2번 검색 결과

distance: 0.48736464977264404 

장소명: 이응팩토리
카테고리: 기타
장소 키워드:  “부여선화핸즈 백제감성체험”
위치: 충남 부여군 규암면 수북로 57
해시태그: 충남, 부여군, 부여, 기타, 추천 여행지, 놀만한 곳, 가볼 만한 곳


- 3번 검색 결과

distance: 0.4853290319442749 

장소명: 짭쪼름갱갱
카테고리: 기타
장소 키워드:  짭쪼름 갱갱
위치: 충남 논산시 강경읍 금백로 135 1층
해시태그: 충남, 논산시, 논산, 기타, 추천 여행지, 놀만한 곳, 가볼 만한 곳


- 4번 검색 결과

distance: 0.46987277269363403 

장소명: 엘캄포
카테고리: 체험
장소 키워드:  열풍 건조 쪽파 디저트 및 스낵
위치: 충남 예산군 신암면 오산리 38-4
해시태그: 충남, 예산군, 예산, 체험, 추천 여행지, 놀만한 곳, 가볼 만한 곳


- 5번 검색 결과

distance: 0.427793025970459 

장소명: 다스름
카테고리: 상점
장소 키워드:  “지역 특산품 밤 율피의 폴리페놀(탄닌)을 이용한 버섯재배“
위치: 충남 부여군 부여읍 계백로 309
해시태그: 충남, 부여군, 부여, 상점, 추천 여행지, 놀만한 곳, 가볼 만한 곳


- 6번 검색 결과

distance: 0.4197462201118469 

장소명: 퍼즐랩
카테고리: 상점
장소 키워드:  “로컬크리에이터 푸드 마켓 플랫폼”
위치: 충남 공주시 먹자1길 5
해시태그: 충남, 공주시, 공주, 상점, 추천 여행지, 놀만한 곳, 가볼 만한 곳


- 

In [ ]:
formatted_results = format_results(results_localCreator, results_nowLocal, results_nature)
formatted_results

"장소명: 민들레역사문화연구소 협동조합\n카테고리: 기타\n장소 키워드:  “랜선으로 만나는 역사 체험 키트 개발”\n위치: 충남 아산시 배방읍 배방로 22 301호\n해시태그: 충남, 아산시, 아산, 기타, 추천 여행지, 놀만한 곳, 가볼 만한 곳\n운영정보: '정보가 없습니다.'\n\n장소명: 이응팩토리\n카테고리: 기타\n장소 키워드:  “부여선화핸즈 백제감성체험”\n위치: 충남 부여군 규암면 수북로 57\n해시태그: 충남, 부여군, 부여, 기타, 추천 여행지, 놀만한 곳, 가볼 만한 곳\n운영정보: '정보가 없습니다.'\n\n장소명: 짭쪼름갱갱\n카테고리: 기타\n장소 키워드:  짭쪼름 갱갱\n위치: 충남 논산시 강경읍 금백로 135 1층\n해시태그: 충남, 논산시, 논산, 기타, 추천 여행지, 놀만한 곳, 가볼 만한 곳\n운영정보: '월~금 10:00 ~ 18:00'\n\n장소명: 엘캄포\n카테고리: 체험\n장소 키워드:  열풍 건조 쪽파 디저트 및 스낵\n위치: 충남 예산군 신암면 오산리 38-4\n해시태그: 충남, 예산군, 예산, 체험, 추천 여행지, 놀만한 곳, 가볼 만한 곳\n운영정보: '정보가 없습니다.'\n\n장소명: 다스름\n카테고리: 상점\n장소 키워드:  “지역 특산품 밤 율피의 폴리페놀(탄닌)을 이용한 버섯재배“\n위치: 충남 부여군 부여읍 계백로 309\n해시태그: 충남, 부여군, 부여, 상점, 추천 여행지, 놀만한 곳, 가볼 만한 곳\n운영정보: '월: 11:00 - 22:00\n화: 11:00 - 22:00\n수: 11:00 - 22:00\n목: 11:00 - 22:00\n금: 11:00 - 22:00\n토: 11:00 - 20:00\n일: 정보없음'\n\n장소명: 퍼즐랩\n카테고리: 상점\n장소 키워드:  “로컬크리에이터 푸드 마켓 플랫폼”\n위치: 충남 공주시 먹자1길 5\n해시태그: 충남, 공주시, 공주, 상점, 추천 여행지, 놀만한 곳, 가볼 만한 곳\n운영정보: '월,화,수,금,토,일 12:00 ~ 19:00 휴무일 목

In [ ]:
LLM_response = sendLLM(example_question, formatted_results)
LLM_response

'충남에서 추천하는 5곳의 여행지를 소개합니다:\n\n1. **민들레역사문화연구소 협동조합**\n   - **위치:** 충남 아산시 배방읍 배방로 22 301호\n   - **카테고리:** 기타\n   - **설명:** "랜선으로 만나는 역사 체험 키트 개발"을 통해 역사 체험을 즐길 수 있는 곳입니다.\n   - **운영정보:** 정보가 없습니다.\n   \n2. **대천해수욕장**\n   - **위치:** 충남 보령시 신흑동 1029\n   - **카테고리:** 자연명소\n   - **설명:** 해변과 산책을 즐길 수 있는 인기 있는 해수욕장입니다.\n   - **운영정보:** 정보가 없습니다.\n\n3. **이응팩토리**\n   - **위치:** 충남 부여군 규암면 수북로 57\n   - **카테고리:** 기타\n   - **설명:** 부여선화핸즈 백제감성체험을 할 수 있는 체험 공간입니다.\n   - **운영정보:** 정보가 없습니다.\n\n4. **계룡산국립공원**\n   - **위치:** 충남 공주시 계룡산로 920\n   - **카테고리:** 자연명소\n   - **설명:** 등산과 경치를 즐길 수 있는 국립공원으로 자연의 아름다움을 만끽할 수 있습니다.\n   - **운영정보:** 정보가 없습니다.\n\n5. **짭쪼름갱갱**\n   - **위치:** 충남 논산시 강경읍 금백로 135 1층\n   - **카테고리:** 기타\n   - **설명:** 현지의 짭쪼름한 특산품과 문화를 체험할 수 있는 공간입니다.\n   - **운영정보:** 월~금 10:00 ~ 18:00\n\n이곳들은 다양한 체험과 아름다운 자연경관을 제공하여 여러분의 여행에 특별한 경험을 선사할 것입니다. 즐거운 충남 여행 되세요!'